In [1]:
%pip install -qU langchain-pinecone pinecone-notebooks langchain pinecone-client google-generativeai openai tqdm chromadb langchain_community langchain-google-genai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.8 MB/s eta 0:00:

In [2]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

Enter your Pinecone API key: ··········


In [3]:
import time

index_name = "langchain-test-index"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [4]:
pip install -qU langchain-google-vertexai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.7 MB/s eta 0:00:00


In [6]:
!pip install google-cloud-aiplatform --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.4 MB/s eta 0:00:00


In [7]:
from google.colab import auth
auth.authenticate_user()

In [9]:
from google.colab import userdata
key=userdata.get('GOOGLE_API_KEY')

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat_model = ChatGoogleGenerativeAI(google_api_key=key,
                                   model="gemini-1.5-flash")

In [12]:
import vertexai

# Replace 'your-project-id' with your actual Google Cloud Project ID
vertexai.init(project="gen-lang-client-0765229835", location="us-central1")

from langchain_google_vertexai import VertexAIEmbeddings

embeddings = VertexAIEmbeddings(model="text-embedding-004")

In [13]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [15]:
!pip install pypdf  # Install if you haven't already
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/docker-deep-dive-nigel-poulton 2024.pdf")
documents = loader.load()

In [16]:
documents

[Document(metadata={'source': '/content/docker-deep-dive-nigel-poulton 2024.pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': '/content/docker-deep-dive-nigel-poulton 2024.pdf', 'page': 1}, page_content='Docker Deep Dive\nZero to Docker in a single book!\nNigel Poulton\nThis book is for sale athttp://leanpub.com/dockerdeepdive\nThis version was published on 2024-05-21\nISBN 9781916585133\nThis is aLeanpub book. Leanpub empowers authors and publishers with the Lean\nPublishing process.Lean Publishingis the act of publishing an in-progress ebook using\nlightweight tools and many iterations to get reader feedback, pivot until you have the\nright book and build traction once you do.\n© 2016 - 2024 Nigel Poulton'),
 Document(metadata={'source': '/content/docker-deep-dive-nigel-poulton 2024.pdf', 'page': 2}, page_content='Huge thanks to my wife and kids for putting up with a geek in the house who genuinely thinks\nhe’s a bunch of software running inside of a container on top 

In [17]:
pages = [doc.page_content for doc in documents]

In [18]:
pages

['',
 'Docker Deep Dive\nZero to Docker in a single book!\nNigel Poulton\nThis book is for sale athttp://leanpub.com/dockerdeepdive\nThis version was published on 2024-05-21\nISBN 9781916585133\nThis is aLeanpub book. Leanpub empowers authors and publishers with the Lean\nPublishing process.Lean Publishingis the act of publishing an in-progress ebook using\nlightweight tools and many iterations to get reader feedback, pivot until you have the\nright book and build traction once you do.\n© 2016 - 2024 Nigel Poulton',
 'Huge thanks to my wife and kids for putting up with a geek in the house who genuinely thinks\nhe’s a bunch of software running inside of a container on top of midrange biological hardware. It\ncan’t be easy living with me!\nMassive thanks as well to everyone who watches my Pluralsight videos. I love connecting with you\nand really appreciate all the feedback I’ve gotten over the years. This was one of the major reasons\nI decided to write this book! I hope it’ll be an ama

In [19]:
from langchain.text_splitter import CharacterTextSplitter  # Import CharacterTextSplitter

# Initialize the CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# Split the text into chunks
chunks = text_splitter.create_documents(pages)

# Display results
print(f"Number of chunks: {len(chunks)}")
print(f"Type of first chunk: {type(chunks[0])}")

Number of chunks: 279
Type of first chunk: <class 'langchain_core.documents.base.Document'>


In [20]:
chunks [7]

Document(metadata={}, page_content='0: About the book 2\nChapter breakdown\n• Chapter 1:Summarises the history and potential future of Docker and containers\n• Chapter 2:Explains the most important container-related standards and projects\n• Chapter 3:Shows you a few ways to get Docker\n• Chapter 4:Walks you through a very simple hands-on container workflow\n• Chapter 5:Explains the architecture of the Docker Engine\n• Chapter 6:Dives deep into images and image management\n• Chapter 7:Dives deep into containers and container management\n• Chapter 8:Walks you through the process of containerizing an app\n• Chapter 9:Shows you how to build, deploy, and manage multi-container apps\nwith Compose\n• Chapter 10:Walks you through building a secure swarm\n• Chapter 11:Deploys and manages a multi-container app on a secure swarm\n• Chapter 12:Walks you through building and containerizing a WebAssembly app\n• Chapter 13:Dives into Docker networking\n• Chapter 14:Builds and tests Docker overlay ne

In [21]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model = GoogleGenerativeAIEmbeddings(google_api_key=key, model="models/embedding-004")


In [22]:
embedding_model = GoogleGenerativeAIEmbeddings(google_api_key=key,  model="models/text-embedding-004",  chunk_size=100,task_type="retrieval_document")


In [23]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

print(type(retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [24]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

In [25]:
chat_template = ChatPromptTemplate.from_messages([
    # System Message Prompt Template
    SystemMessage(content="""You are a Helpful AI Bot.
                  Given a context and question from user,
                  you should answer based on the given context."""),
    # Human Message Prompt Template
    HumanMessagePromptTemplate.from_template("""Answer the question based on the given context.
    Context: {context}
    Question: {question}
    Answer: """)
])

In [26]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [27]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | chat_template
    | chat_model
    | output_parser
)

In [28]:
response = rag_chain.invoke("""Please summarize Leave No Context Behind:
                            Efficient Infinite Context Transformers with Infini-attention""")

response

'The paper "Leave No Context Behind: Efficient Infinite Context Transformers with Infini-attention" introduces a new transformer architecture designed to handle arbitrarily long sequences.  It achieves this through a novel attention mechanism called "Infini-attention," which avoids the quadratic complexity typically associated with long sequences in standard transformers.  Instead of computing attention over the entire sequence, Infini-attention uses a more efficient approach that allows the model to effectively attend to all relevant context, regardless of length, making it suitable for tasks requiring very long-range dependencies.  The key is improved efficiency without sacrificing performance on long sequences.\n'

In [29]:
response = retriever.invoke("""Please summarize Leave No Context Behind:
                            Efficient Infinite Context Transformers with Infini-attention""")

response

[]

In [30]:
response = rag_chain.invoke("""Please summarize Chapter 1:""")

response

"Please provide the text of Chapter 1.  I need the chapter's content to summarize it.\n"

In [31]:
response = rag_chain.invoke("""Who are you?""")

response

'I am a large language model, trained by Google.\n'

In [32]:
response = rag_chain.invoke("""What can you help me?""")

response

'I can help you answer questions based on the provided context.  If no context is given, I can try my best to answer your question using my general knowledge.\n'

In [33]:
response = rag_chain.invoke("""Is pine cone setup in this colab?""")

response

'The provided context is empty.  Therefore, I cannot answer whether a pine cone setup is in the colab.  More context is needed.\n'

In [35]:
response = rag_chain.invoke("""What is Gemini Flash optimized for?""")

response

'The provided context is empty.  Therefore, I cannot answer the question about what Gemini Flash is optimized for.  I need context about Gemini Flash to provide an answer.\n'

In [39]:
response = rag_chain.invoke("""What is the capital of USA?""")

response

'The provided context does not contain the answer to the question "What is the capital of the USA?".  Therefore, I cannot answer the question based on the given context.\n'